In [16]:
# import pandas library to read data from files and to create data frames
import pandas as pd

# read the entitiy-article relationship data from csv file to a dataframe
entityRel = pd.read_csv('data_files/entityRel.csv',sep=';')

# print the column names and the dataframe dimensions
print(entityRel.columns)
print(entityRel.shape)

Index([u'id', u'relevance', u'date_added', u'article_id', u'entity_id'], dtype='object')
(19377, 5)


In [17]:
# Get rid of the keyword-article relations that has 0 relevance.
# We do that because 0 relevance means we're not sure at all whether that keyword has a relationship with the keyword or not

entityRelNotZero = entityRel[entityRel['relevance'] > 0]

In [27]:
keywordsList = []

# We have 908 different article for the current data so we will run a for loop 908 times

for num in range(1, 909):
    # This is how we will get the keywords from the same article
    similarKeywords = entityRelNotZero[entityRelNotZero['article_id'] == num]

    # create a link between all keyword from the same article
    for x in range(1, similarKeywords.shape[0]):
        keywordsList.append({'source': similarKeywords.iloc[0]['entity_id'], 'target': similarKeywords.iloc[x]['entity_id'], 'value': int(similarKeywords.iloc[0]['relevance']*similarKeywords.iloc[x]['relevance']*100)})
            

In [28]:
keywordsLinksCSV = open('data_files/keywordsLinksSmallLarge.csv', 'w')

keywordsLinksCSV.write('"source";"target";"value"\n')
for item in keywordsList:
    keywordsLinksCSV.write(str(item['source']) + ';' + str(item['target']) + ';' + str(item['value']) + '\n')

In [29]:
# read the csv for links between keywords
linksBetweenKeywords = pd.read_csv('data_files/keywordsLinksSmallLarge.csv',sep=';')

# print the column names and the dataframe dimensions
print(linksBetweenKeywords.columns)
print(linksBetweenKeywords.shape)

Index([u'source', u'target', u'value'], dtype='object')
(17967, 3)


In [30]:
# create keyword nodes
# different articles may include same keywords
# we need to be sure that we don't repeat the same keyword as node
# so we use sets to add keywords
keywordNodes = set([])
for x in range(0, entityRelNotZero.shape[0]):
    keywordNodes.add(entityRelNotZero.iloc[x]['entity_id'])

In [32]:
# Create JSON file for nodes and links
keywordsLinksJSON = open('data_files/keywordsLinksSmallLarge.json', 'w')

# Add nodes to json file
keywordsLinksJSON.write('{\n')
keywordsLinksJSON.write('"nodes":[\n')

for node in keywordNodes:
    keywordsLinksJSON.write('{"id": ' + str(node) + ', "group": ' + str(1) + '},\n')

keywordsLinksJSON.write('],\n')
keywordsLinksJSON.write('"links": [\n')

for item in keywordsList:
    keywordsLinksJSON.write('{"source": ' + str(item['source']) + ', "target": ' + str(item['target']) + ', "value": ' + str(item['value']) + '},\n')

keywordsLinksJSON.write(']\n')
keywordsLinksJSON.write('}\n')